In [27]:
import numpy as np
import pandas as pd

import time
from pathos.multiprocessing import ProcessingPool as Pool
# from multiprocessing import Pool
from tqdm.notebook import tqdm
import hashlib
import json
import glob
import urllib
import urllib.request as urllib2
import requests
import http
import langid

from goose3 import Goose
from goose3.text import StopWordsChinese, StopWords, StopWordsArabic, StopWordsKorean

import warnings
warnings.filterwarnings('ignore')
print('The script still runs.')


The script still runs.


# event data download


In [28]:
import os
import zipfile
import requests
import time
import multiprocessing
from bs4 import BeautifulSoup
from datetime import datetime

path = os.path.split(os.path.abspath('__file__'))[0] + os.sep
data_folder = path + '/event_data' + os.sep
base_url = 'http://data.gdeltproject.org/events/'

def get_url_list(page_url):
    html = requests.get(page_url).text
    soup = BeautifulSoup(html, 'lxml')
    url_list = []
    for li in soup.body.ul.find_all('li'):
        href = base_url + li.a['href']
        url_list.append(href)
    with open(path + 'url_list.txt', 'w') as f:
        for url in url_list[3:-1]:
            f.write(url + '\n')
    return url_list[3:-1]


def get_url_list_by_span(page_url, begin=20130401, end=20220101):
    url_list_by_span = []
    url_list = get_url_list(page_url)
    for url in url_list:
        filename = url.split('/')[-1]
        date = filename.split('.')[0]
        length = len(str(date))
        if length <= 6:
            if int(str(begin)[:length]) <= int(date) <= int(str(end)[:length]):
                url_list_by_span.append(url)
        else:
            if begin <= int(date) <= end:
                url_list_by_span.append(url)
    with open(path + 'url_list.txt', 'w') as f:
        for url in url_list_by_span:
            f.write(url + '\n')
    return url_list_by_span


def get_url_data(url):
    if not os.path.exists(data_folder):
        os.mkdir(data_folder) 
    filename = url.split('/')[-1]
    filepath = data_folder + filename
    if os.path.exists(filepath) or os.path.exists(filepath[:-4]):
        return
    try:
        data = requests.get(url)
        with open(filepath, "wb") as f:
            f.write(data.content)
        fz = zipfile.ZipFile(filepath, 'r')
        fz.extract(fz.namelist()[0], data_folder)
        fz.close()
        if os.path.exists(filepath):
            os.remove(filepath)
    except Exception as e:
        print(e)
        log = open(path + 'log.txt', 'a')
        log.write(url + '\n')


def main():
    t1 = time.time()
    begin = 20150101
    end = 20240601
    url = base_url + 'index.html'
    url_list = get_url_list_by_span(url, begin, end)
    for i in url_list:
        get_url_data(i)

if __name__ ==# extract and merge GKG data '__main__':
    main()


# extract and event GKG data

In [ ]:
import os
import zipfile
import pandas as pd
from datetime import datetime
import glob
import numpy as np
import csv

# Define the path to the directory containing the zip files
input_dir = "Download/"
output_dir = "event_data/"  # Define your output directory here

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to extract and read CSV files from a zip file
def extract_and_read_csv(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file in zip_ref.namelist():
            if file.endswith('.csv'):
                with zip_ref.open(file) as csvfile:
                    df = pd.read_csv(csvfile, on_bad_lines ='skip', encoding = "ISO-8859-1", encoding_errors=None, sep='\t',
                                     quoting=csv.QUOTE_NONE, header=None)
                    return df
    return None

def merge(g):
    print(g)
    merged_df = pd.DataFrame()
    group_files = glob.glob(input_dir + '\*' + g + '*.event.csv.zip')
    for f in group_files:
        df = extract_and_read_csv(f)
        merged_df = pd.concat([merged_df, df], ignore_index=True)
    output_filename = os.path.join(output_dir, f"{g[:4]}-{g[4:6]}-{g[6:8]}")
    merged_df.to_csv(output_filename, index=False, sep='\t')

# import multiprocessing as mp
# pool = mp.Pool()
# res = pool.map(merge, group)

files = glob.glob(input_dir+'\\*.event.csv.zip')
group = np.unique([i[8:16] for i in files])
for g in group:
    merge(g)


# event data screening

In [38]:
df1 = pd.read_excel('SPGlobal_general_info.xlsx')
df2 = pd.read_excel('SPGlobal_loc_risk.xlsx')
df = pd.merge(df1, df2, on=['prop_name', 'prop_id'])

df['lonlat'] = df['longitude'].map(lambda x: str(np.round(x, 0))) + '|' + df['latitude'].map(lambda x: str(np.round(x, 0)))
df = df.dropna(subset=['lonlat'])
df.head()


,prop_name,prop_id,prop_id.1,prop_name.1,also_known_as,primary_commodity,dev_stage,actv_status,yr_source_date,mo_source_date,...,coordinate_accuracy,distance_from,location_comments,locale,district,country/region_pol_risk,country/region_op_risk,country/region_sec_risk,country/region_ter_risk,lonlat
0,Chihuahua Refinery,86959,86959,Chihuahua Refinery,NaN,Lead,Operating,Active,2020.0,2.0,...,Exact,25 km SW of Juan Aldama,NaN,NaN,NaN,Medium,Medium,Medium,Low,-106.0|29.0
1,101,64665,64665,101,NaN,Coal,Operating,NaN,2011.0,12.0,...,Exact,140 km SSW of Florianópolis,NaN,NaN,NaN,Medium,Medium,Medium,Low,-49.0|-29.0
2,1068,79522,79522,1068,NaN,Gold,Exploration,Active,2020.0,10.0,...,Best Guess,NaN,NaN,NaN,NaN,Medium,Medium,Medium,Low,-109.0|29.0
3,10th of Ramadan Refinery,86452,86452,10th of Ramadan Refinery,NaN,Lead,Operating,Active,2021.0,2.0,...,Exact,60 km NE of Cairo,NaN,NaN,NaN,Medium,Medium,Medium,Medium,32.0|30.0
4,10th of Ramadan Smelter,86451,86451,10th of Ramadan Smelter,NaN,Lead,Operating,Active,2021.0,2.0,...,Exact,60 km NE of Cairo,NaN,NaN,NaN,Medium,Medium,Medium,Medium,32.0|30.0


In [39]:
ETMs = [
       'Copper', 'Lead', 
       'Palladium', 'Platinum', 'Platinum Group Metals',
       'Nickel', 'Vanadium',
       'Cobalt', 'Lithium',
       'Graphite', 'Manganese', 'Molybdenum',
       'Bauxite',# 'Alumina', 'Aluminum',
       'Chromium'
        ]

df['is_ETM'] = (df['primary_commodity'].isin([i for i in ETMs])).astype(int)
df = df[df['is_ETM']==1]
df.head()


,prop_name,prop_id,prop_id.1,prop_name.1,also_known_as,primary_commodity,dev_stage,actv_status,yr_source_date,mo_source_date,...,distance_from,location_comments,locale,district,country/region_pol_risk,country/region_op_risk,country/region_sec_risk,country/region_ter_risk,lonlat,is_ETM
0,Chihuahua Refinery,86959,86959,Chihuahua Refinery,NaN,Lead,Operating,Active,2020.0,2.0,...,25 km SW of Juan Aldama,NaN,NaN,NaN,Medium,Medium,Medium,Low,-106.0|29.0,1
3,10th of Ramadan Refinery,86452,86452,10th of Ramadan Refinery,NaN,Lead,Operating,Active,2021.0,2.0,...,60 km NE of Cairo,NaN,NaN,NaN,Medium,Medium,Medium,Medium,32.0|30.0,1
4,10th of Ramadan Smelter,86451,86451,10th of Ramadan Smelter,NaN,Lead,Operating,Active,2021.0,2.0,...,60 km NE of Cairo,NaN,NaN,NaN,Medium,Medium,Medium,Medium,32.0|30.0,1
6,113 North,87482,87482,113 North,NaN,Nickel,Target Outline,Active,2021.0,6.0,...,35 km SSW of Lebel-sur-Quévillon,NaN,NaN,NaN,Low,Low,Low,Low,-77.0|49.0,1
15,1514-M,29336,29336,1514-M,NaN,Nickel,Target Outline,Inactive,1995.0,9.0,...,80 km N of Voisey Bay; 55 km SW of Nutak,NaN,NaN,NaN,Low,Low,Low,Low,-62.0|55.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39111,Zulu,41737,41737,Zulu,"Acorn, Altyr, Cakewalk, Deep Purple, Main Zone...",Lithium,Feasibility Started,Active,2021.0,8.0,...,55 km NNE of Filabusi,NaN,NaN,NaN,High,High,Medium,Insignificant,29.0|-20.0,1
39115,Zunyi,60815,60815,Zunyi,NaN,Bauxite,Operating,Active,2021.0,4.0,...,NaN,NaN,NaN,NaN,Medium,Medium,Low,Low,107.0|28.0,1
39118,Zuun Mod,35497,35497,Zuun Mod,"Black Cirque, Intersection Zone, Khuvyn Khar, ...",Molybdenum,Reserves Development,Active,2021.0,8.0,...,270 km SSW of Bayankhongor,NaN,NaN,NaN,Medium,Medium,Medium,Insignificant,99.0|44.0,1
39128,Zwemkloof,67186,67186,Zwemkloof,NaN,Platinum,Exploration,Inactive,2013.0,7.0,...,NaN,NaN,NaN,NaN,Medium,High,Medium,Low,29.0|-24.0,1


In [42]:
df['primary_commodity'].value_counts()


Copper                   5041
Nickel                   1399
Lead                     1127
Lithium                   457
Platinum                  340
Graphite                  313
Molybdenum                300
Bauxite                   240
Manganese                 213
Cobalt                    145
Vanadium                  102
Palladium                  29
Chromium                    4
Platinum Group Metals       1
Name: primary_commodity, dtype: int64

In [6]:
def haversine_funct(lat1, lon1, lat2, lon2):
    # Calculates the great circle distance between two locations given in decimal degrees latitude and logitude
    kms = 3959*1.60934
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    total_km = kms * c
    return total_km

haver_vec = np.vectorize(haversine_funct)                      # vectorize the haversine function for speed

def haver_subset(df, y):
    '''Filters a dataframe based on a location, search radius, and event codes
    Inputs:
        df: a dataframe containing GDELT search results
        x: a latitude, longitude tuple
        y: a distance in kilometers
        z: a list of event codes
    Returns a filtered dataframe'''
    df['Dist'] = haver_vec(df['latitude'].values, df['longitude'].values, df['ActionGeo_Lat'].values, df['ActionGeo_Long'].values) # calc distance
    df = df[df['Dist'] < y]                                    # filter by distance
    return df


In [8]:
import warnings
from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
warnings.filterwarnings("ignore")


In [11]:
files = glob.glob('event_data/20*')
col = [ 'GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type', 'ActionGeo_FullName', 'ActionGeo_CountryCode', 'ActionGeo_ADM1Code', 'ActionGeo_Lat', 'ActionGeo_Long', 'ActionGeo_FeatureID', 'DATEADDED', 'SOURCEURL'] 
col2 = ['GLOBALEVENTID', 'MonthYear', 'Year', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1Type1Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2Type1Code', 'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_FullName', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type', 'ActionGeo_FullName', 'ActionGeo_CountryCode', 'ActionGeo_ADM1Code', 'ActionGeo_Lat', 'ActionGeo_Long', 'ActionGeo_FeatureID', 'DATEADDED', 'SOURCEURL']
failed = []
for f in files:
    try:
        data = pd.read_csv(f, sep='\t', error_bad_lines=False, header=None)
        data.columns = col
        data = data[col2]
        data['lonlat'] = data['ActionGeo_Long'].map(lambda x: str(np.round(x, 0))) + '|' + data['ActionGeo_Lat'].map(lambda x: str(np.round(x, 0)))
        data = pd.merge(data, df, on='lonlat', how='inner')

    #     print(data.shape)
        dist = 100                                                     # user specified search distance in km
        data = haver_subset(data, dist)       # call the filtering function

    #     print(data.shape)
        data.to_csv(f.replace('data', 'event_mining_data'), index=None)
    except:
        print(f)
        failed += [f]


data\20161004.export.CSV
data\20170106.export.CSV
data\20170422.export.CSV
data\20170802.export.CSV
data\20170920.export.CSV
data\20171021.export.CSV
data\20180415.export.CSV
data\20180416.export.CSV
data\20180613.export.CSV
data\20190217.export.CSV
